In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import sympy as sp
from tabulate import tabulate
%matplotlib inline

In [67]:
def solve_lin_system(a, b, c, f):
    """Решение СЛАУ методом прогонки"""

    n = len(c)
    x = [sp.Rational(0) for i in range(n)]
    alpha = [sp.Rational(0) for i in range(n - 1)]
    betta = [sp.Rational(0) for i in range(n)]
    # Прямая прогонка
    alpha[0] = -b[0] / c[0]
    betta[0] = f[0] / c[0]
    for i in range(1, n - 1):
        alpha[i] = -b[i] / (c[i] - alpha[i - 1] * (-a[i - 1]))
        betta[i] = (f[i] - a[i - 1] * betta[i - 1]) / (c[i] + a[i - 1] * alpha[i - 1])
    betta[-1] = (f[-1] - a[-1] * betta[-2]) / (c[-1] + a[-1] * alpha[-1])

    # Обратная прогонка
    x[-1] = betta[-1]
    for i in reversed(range(n - 1)):
        x[i] = alpha[i] * x[i + 1] + betta[i]
    print('::::::::::::::::::::::::::::::::::::::::::ПРОГОНКА::::::::::::::::::::::::::::::::::::::::::')
    print('alpha:::::::::::::::::::::',alpha)
    print('betta:::::::::::::::::::::',betta)
    print('x:::::::::::::::::::::',x)
    return np.asarray(x)


def newton_pol(n,func,nodes):
    y = np.full((n+1,n+2),sp.Rational(0))
    for j in range(n+1):
        y[j][0] = sp.nsimplify(func(nodes[j]), tolerance=0.001, rational=True)
    for h in range(n-1,-1,-1):
        k = n-h
        for i in range(0,h+1):
            y[i][k] = (y[i+1][k-1]-y[i][k-1])/(nodes[i+k]-nodes[i])
    print(tabulate(y,tablefmt="pretty", floatfmt=".5f"))
    def function(x):
        w = np.cumprod(sp.nsimplify(x-nodes, tolerance=0.001, rational=True))
        result = y[0][0]+np.sum(np.dot(w,y[0,1:]))
        print('coefs of newton polynomial:\n',y[0][0],np.sum(np.dot(w,y[0,1:])),sep='\t')
        return result, abs(func(x)-result)
    return function


def get_nodes(a,b,n,chebyshev=False):
    nodes = np.full(n+1,sp.Rational(0))
    if chebyshev:
        for i in range(n+1):
            nodes[i] = (a+b)/2 + (b-a)/2*math.cos(math.pi*(2*i+1)/(2*n+2))
    else:
        h = (b-a)/(n)
        nodes[0] = a
        for i in range(1,n+1):
            nodes[i] = nodes[i-1]+h
    print('nodes:')
    for i in range(n+1):
        print(f'x_{i}=',nodes[i],end='\t')
    return nodes


def upper_bound(array, key):
    left=0
    right = len(array)-1
    while left < right:
        mean = (left+right)//2
        if key<array[mean]:
            right = mean 
        if key>array[mean]:
            left = mean + 1
        if key == array[mean]:
            return mean+1
    return mean        
    
    
def get_spline(gamma):
    delta = np.zeros((n))
    betta = np.zeros((n))    
    alpha = values[1:]
    for i in range(n):
        delta[i] = (gamma[i+1]-gamma[i])/h_i[i]
        betta[i] = (values[i+1]-values[i])/h_i[i]+(2*gamma[i+1]+gamma[i])*h_i[i]/6
    print('::::::::::::::::::::::::::::::::::::::::::SPLINE::::::::::::::::::::::::::::::::::::::::::')
    print('delta:::::::::::::::::::::',delta)
    print('betta:::::::::::::::::::::',betta)
    print('alpha:::::::::::::::::::::',alpha)
    def spline(x):
        idx = upper_bound(nodes,x)
        return alpha[idx]+betta[idx]*(x-nodes[idx+1])+gamma[idx+1]/2*(x-nodes[idx+1])**2+delta[idx]/6*(x-nodes[idx+1])**3
        
    return spline

In [99]:
def func(x):
    return math.sin(x+1/2)
a,b = sp.Rational(0),sp.Rational(math.pi/2)
n = 3
h = (b-a)/n
sfunc = sp.cos(t+t**2)
drvtv1 = sp.lambdify(t,sp.diff(sfunc,t))
drvtv2 = sp.lambdify(t,sp.diff(sfunc,t,t))

nodes = np.array([0,1,2,4])
vfunc = np.vectorize(func)
values = np.array([2,3,5,10])
coefs = np.zeros((n+1,n+1))
column = np.zeros((n+1,1))
h_i = np.array([sp.nsimplify(nodes[i+1]-nodes[i], tolerance=0.001,rational=True) for i in range(n)])
differences = np.array([sp.nsimplify((values[i+1]-values[i])/h_i[i], tolerance=0.001,rational=True) for i in range(n)])
for i in range(1,n):
    coefs[i][i-1] = h_i[i-1]/(h_i[i-1]+h_i[i])
    coefs[i][i] = sp.Rational(2)
    coefs[i][i+1] = h_i[i]/(h_i[i-1]+h_i[i])
    column[i] = 6*(differences[i]-differences[i-1])/(h_i[i-1]+h_i[i])

In [100]:
# natural case
print(coefs)
print(column)
a_coef = [sp.nsimplify(-coefs[i+1][i], tolerance=0.001,rational=True) for i in range(1,n)]
c_coef = [sp.nsimplify(coefs[i][i], tolerance=0.001,rational=True) for i in range(1,n)]
b_coef = [sp.nsimplify(-coefs[i][i+1], tolerance=0.001,rational=True) for i in range(1,n)]
print('a:', a_coef,'\nc:',c_coef,'\nb:',b_coef)
gamma1 = solve_lin_system(a_coef,b_coef,c_coef,column[1:-1])
gamma1 = np.insert(gamma1,0,0)
gamma1 = np.insert(gamma1, n,0)
spline1 = get_spline(gamma1)
print('gamma:',gamma1)
newton_pol(n,func,get_nodes(a,b,n,chebyshev=True))

[[0.         0.         0.         0.        ]
 [0.5        2.         0.5        0.        ]
 [0.         0.33333333 2.         0.66666667]
 [0.         0.         0.         0.        ]]
[[0.]
 [3.]
 [1.]
 [0.]]
a: [-1/3, 0] 
c: [2, 2] 
b: [-1/2, -2/3]
::::::::::::::::::::::::::::::::::::::::::ПРОГОНКА::::::::::::::::::::::::::::::::::::::::::
alpha::::::::::::::::::::: [1/4]
betta::::::::::::::::::::: [array([1.50000000000000], dtype=object), array([0.500000000000000], dtype=object)]
x::::::::::::::::::::: [1.62500000000000, array([0.500000000000000], dtype=object)]
::::::::::::::::::::::::::::::::::::::::::SPLINE::::::::::::::::::::::::::::::::::::::::::
delta::::::::::::::::::::: [ 1.625 -1.125 -0.25 ]
betta::::::::::::::::::::: [1.54166667 2.4375     2.66666667]
alpha::::::::::::::::::::: [ 3  5 10]
gamma: [0 1.62500000000000 array([0.500000000000000], dtype=object) 0]
nodes:
x_0= 1.51101145143231	x_1= 1.08595702833962	x_2= 0.484839298455275	x_3= 0.0597848753625906	-------  -----

<function __main__.newton_pol.<locals>.function(x)>

### first order drvtv case

coefs[0][0] = 2
coefs[0][1] = 1
column[0] = 6/h_i[0]*(differences[0]-drvtv1(a))

coefs[n][n-1] = 1
coefs[n][n] = 2
column[n] = 6/h_i[n-1]*(drvtv1(b)-differences[n-1])
a_coef = [-coefs[i+1][i] for i in range(n+1)]
c_coef = [coefs[i][i] for i in range(n+1)]
b_coef = [-coefs[i][i+1] for i in range(n+1)]
print('a:', a_coef,'\nc:',c_coef,'\nb:',b_coef)
gamma1 = solve_lin_system(a_coef,b_coef,c_coef,column[1:-1])
gamma1 = np.insert(gamma1,0,0)
gamma1 = np.insert(gamma1, n,0)
spline1 = get_spline(gamma1)
print('gamma:',gamma1)
newton_pol(n,func,get_nodes(a,b,n))
gamma1 = np.linalg.solve(coefs,column)
# alpha1, betta1, delta1 = get_rest_coefs(gamma1)
spline1 = get_spline(gamma1)


In [66]:
sp.nsimplify(2.45, tolerance=0.001,rational=True)